In [25]:
# get data, remove newline
import os

file_dict = {}
files = sorted(os.listdir("./languageID"))
for file in files:
    f = open(os.path.join("./languageID",file), "r")
    content = f.read()
    f.close()
    file_dict.update({file:content})

for k, v in file_dict.items():
    file_dict[k] = ''.join(v.splitlines())

In [110]:
# Q3-1 prior probability
import math

languages = 'ejs'
smoothing_param = 0.5
traing_idx = range(10) #0...9
prior_count = [smoothing_param]*3
for k in file_dict.keys():
    for i,l in enumerate(languages):
        if l in k and int(k.split('.')[0][1:]) in traing_idx:
            prior_count[i] +=1
prior_total_count = sum(prior_count)
prior_log_prob = {l: math.log(count / total_count) for l,count in zip(languages,prior_total_count)}
print([f'P(y={k})= {math.exp(v)} 'for k,v in prior_log_prob.items()])

['P(y=e)= 0.3333333333333333 ', 'P(y=j)= 0.3333333333333333 ', 'P(y=s)= 0.3333333333333333 ']


In [68]:
def word_count(string):
    word = 'abcdefghijklmnopqrstuvwxyz '
    my_dict = {}
    for c in word:
        my_dict[c] = string.count(c)
    return my_dict

In [140]:
# count occurance
from collections import Counter
word = 'abcdefghijklmnopqrstuvwxyz '
languages = 'ejs'
smoothing_param = 0.5
traing_idx = range(10) #0...9
count_dict = [Counter({c: smoothing_param for c in word}) for _ in languages]
for k, v in file_dict.items():
    for i,l in enumerate(languages):
        if l in k and int(k.split('.')[0][1:]) in traing_idx:
            count_dict[i]+=Counter(word_count(v))



In [161]:
# Q3-2, Q3-3conditional probability

cond_total_count = [sum(dict.values()) for dict in count_dict]
cond_log_prob = {l:  {k:math.log(v/count) for k, v in dict.items()}  for l,count,dict in zip(languages,cond_total_count,count_dict)}

for l in languages:
    print(f'theta_{l}')
    print([f'{math.exp(v):.3f}' for v in cond_log_prob[l].values()])


theta_e
['0.060', '0.011', '0.022', '0.022', '0.105', '0.019', '0.017', '0.047', '0.055', '0.001', '0.004', '0.029', '0.021', '0.058', '0.064', '0.017', '0.001', '0.054', '0.066', '0.080', '0.027', '0.009', '0.015', '0.001', '0.014', '0.001', '0.179']
theta_j
['0.132', '0.011', '0.005', '0.017', '0.060', '0.004', '0.014', '0.032', '0.097', '0.002', '0.057', '0.001', '0.040', '0.057', '0.091', '0.001', '0.000', '0.043', '0.042', '0.057', '0.071', '0.000', '0.020', '0.000', '0.014', '0.008', '0.123']
theta_s
['0.105', '0.008', '0.038', '0.040', '0.114', '0.009', '0.007', '0.005', '0.050', '0.007', '0.000', '0.053', '0.026', '0.054', '0.072', '0.024', '0.008', '0.059', '0.066', '0.036', '0.034', '0.006', '0.000', '0.002', '0.008', '0.003', '0.168']


In [211]:
#Q3-4 
bow = word_count(file_dict['e10.txt'])
log_likelihood = {}
for l in languages:
    log_likelihood.update({l: sum([bow[k] * v for k,v in cond_log_prob[l].items()])})

for l in languages:
    print(f'p(x | y = {l}) = e^({log_likelihood[l]})')

p(x | y = e) = e^(-7841.865447060635)
p(x | y = j) = e^(-8771.433079075032)
p(x | y = s) = e^(-8467.282044010557)


In [213]:
#Q3-5 posterior 
unnorm_log_posterior = {}
for l in languages:
    unnorm_log_posterior.update({l: log_likelihood[l] + prior_log_prob[l]}) #stored in log, so add instead of multiply

for l in languages:
    print(f'p(y = {l}) | x) = e^({unnorm_log_posterior[l]})')

p(y = e) | x) = e^(-7842.964059349303)
p(y = j) | x) = e^(-8772.5316913637)
p(y = s) | x) = e^(-8468.380656299225)


In [245]:
def getLabel(dict):
    maxProb = max(dict.values()) 
    for k,v in dict.items():
        if v == maxProb:
            return k

In [251]:
#Q3-6 test set
y_true = []
y_pred = []
for f in languages:
    for i in range(10,20): #10...19
        bow = word_count(file_dict[f+str(i)+'.txt'])
        log_likelihood = {}
        unnorm_log_posterior = {}
        for l in languages:
            log_likelihood.update({l: sum([bow[k] * v for k,v in cond_log_prob[l].items()])})
            unnorm_log_posterior.update({l: log_likelihood[l] + prior_log_prob[l]}) 
        y_true.append(f)
        y_pred.append(getLabel(unnorm_log_posterior))
print(y_true)
print(y_pred)

['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's']
['e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'e', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 'j', 's', 's', 's', 's', 's', 's', 's', 's', 's', 's']
